In [1]:
from __future__ import unicode_literals
import spacy
nlp = spacy.load('en')

In [53]:
dataset = "valid"

In [54]:
import os
fname = os.getcwd() + "/data/conll2003/en/%s.txt" % dataset
f = open(fname)
content = f.readlines()

In [55]:
def tokenDescr(tok):
    if tok.ent_iob_ == "O":
        return tok.ent_iob_
    if tok.ent_type_ in ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']:
        return "O"
    mapping = {
        "PERSON": "PER",
        "NORP": "LOC",
        "FACILITY": "LOC",
        "ORG": "ORG",
        "GPE": "LOC",
        "LOC": "LOC",
        "PRODUCT": "MISC",
        "EVENT": "MISC",
        "WORK_OF_ART": "MISC",
    }
    if tok.ent_type_ in mapping:
        return tok.ent_iob_ + '-' + mapping[tok.ent_type_]
    else:
        return tok.ent_iob_ + '-MISC'

In [56]:
output = []
ok = 0
ko = 0
def handleSentence(s):
    global ok, ko
    text = ' '.join([o[0] for o in s])
    doc = nlp(text)
    # print(len(s), len(doc))
    if len(s) == len(doc):
        ok += 1
        ####
        ####
        for idx, o in enumerate(s):
            tok = doc[idx]
            output.append(' '.join([o[0], o[-1], tokenDescr(tok)]))
        # End of sentence
        output.append(' ')
    else:
        ko += 1
        # Discard samples with different tokenizations 
    


In [57]:
dummy = []
def dummyOutput(l):
    dummy.append(l)

In [58]:
currSentence = []

for x in content:
    x = x.strip()
    if x.startswith('-DOCSTART-'):
        continue
    if x == '':
        dummyOutput('')
        text = ' '.join([o[0] for o in currSentence])
        handleSentence(currSentence)
        # handle sentence
        currSentence = []
    else:
        objs = x.split(' ')
        # print objs
        dummyOutput(x+' '+objs[-1])
        currSentence.append((objs[0], objs[-1]))
        

In [59]:
ok, ko

(2422, 1044)

In [60]:
def writeToFile(fname, tokens):
    with open(fname, 'w') as f:
        f.write('\n'.join(tokens))


In [61]:
writeToFile('dummy.txt', dummy)

In [62]:
writeToFile("%s-output.txt" % dataset, output)